In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
model = "dhanishetty/flan-t5-base-merged"
tokenizer = AutoTokenizer.from_pretrained(model,trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model, trust_remote_code=True, torch_dtype="auto")

In [3]:
billsum = load_dataset("billsum", split= 'train')

In [4]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 18949
})

In [5]:
inputs = tokenizer(billsum['text'][1], return_tensors="pt", max_length= 1024, truncation=True)
outputs = model.generate(**inputs, max_length = 128)
predictions =  tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(predictions)

Human Rights Information Act - Directs the President to: (1) establish a human rights information and declassification program for the declassification of human rights documents; (2) establish a human rights information and declassification program for the declassification of human rights documents; and (3) establish a human rights information and declassification program for the declassification of human rights documents. Authorizes the President to establish a human rights information and declassification program for the declassification of human rights documents. Authorizes the President to establish a human rights information and declassification program for the declass


In [6]:
references = billsum['summary'][1]
print(references)

Human Rights Information Act - Requires certain Federal agencies to identify and organize all human rights records regarding activities occurring in Guatemala and Honduras after 1944 for declassification and disclosure purposes, and to make them available to the public. 
Instructs the President to report to Congress regarding agency compliance. 
Prescribes guidelines under which the Interagency Security Classification Appeals Panel shall review agency determinations to postpone public disclosure of any human rights record.  Authorizes postponement of such public disclosures on specified grounds. 
Requires any U.S. agency, upon request by an entity created by the United Nations, the Organization of American States (or similar entity), a national truth commission (or similar entity), or from the principal justice or human rights official of a country that is investigating a pattern of gross violations of internationally recognized human rights, to review, declassify, and publicly disclos

In [7]:
def trim_to_match_length(reference_text, prediction_text):
    # Calculate the length of the prediction text
    prediction_length = len(prediction_text)

    # Trim the reference text to match the length of the prediction text
    trimmed_reference_text = reference_text[:prediction_length]

    return trimmed_reference_text

# Example usage
reference_text = references
prediction_text = predictions

references = trim_to_match_length(reference_text, prediction_text)
print(references)


Human Rights Information Act - Requires certain Federal agencies to identify and organize all human rights records regarding activities occurring in Guatemala and Honduras after 1944 for declassification and disclosure purposes, and to make them available to the public. 
Instructs the President to report to Congress regarding agency compliance. 
Prescribes guidelines under which the Interagency Security Classification Appeals Panel shall review agency determinations to postpone public disclosure of any human rights record.  Authorizes postponement of such public disclosures on specified grounds. 
Requires any U.S. agency, upon request by an entity created by the United Nati


In [8]:
import evaluate
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=references, use_aggregator=True)

In [9]:
print(results)

{'rouge1': 0.07331378299120235, 'rouge2': 0.0, 'rougeL': 0.0718475073313783, 'rougeLsum': 0.07331378299120235}


In [10]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

{'bleu': 0.0, 'precisions': [0.08629441624365482, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0119863013698631, 'translation_length': 591, 'reference_length': 584}
